# Creating extensions using numpy and scipy

In this notebook, we shall go through two tasks:

1. Create a neural network layer with no parameters. 
  - This calls into **numpy** as part of it's implementation
2. Create a neural network layer that has learnable weights
  - This calls into **SciPy** as part of it's implementation

In [37]:
import torch
from torch.autograd import Function
from torch.autograd import Variable

## Parameter-less example

This layer doesn't particularly do anything useful or mathematically correct.

It is aptly named BadFFTFunction

**Layer Implementation**

In [38]:
from numpy.fft import rfft2, irfft2
class BadFFTFunction(Function):
    
    def forward(self, input):
        numpy_input = input.numpy()
        result = abs(rfft2(numpy_input))
        return torch.FloatTensor(result)
    
    def backward(self, grad_output):
        numpy_go = grad_output.numpy()
        result = irfft2(numpy_go)
        return torch.FloatTensor(result)

# since this layer does not have any parameters, we can
# simply declare this as a function, rather than as an nn.Module class
def incorrect_fft(input):
    return BadFFTFunction()(input)

**Example usage of the created layer:**

In [39]:
input = Variable(torch.randn(8, 8), requires_grad=True)
result = incorrect_fft(input)
print(result.data)
result.backward(torch.randn(result.size()))
print(input.grad)


  4.7742   8.5149   9.8856  10.2735   8.4410
  3.8592   2.2888   5.0019   5.9478   5.1993
  4.6596   3.4522   5.9725  11.0878   7.8076
  8.2634   6.6598   6.0634  15.5515   6.9418
  0.6407   7.4943   0.8726   4.4138   7.1496
  8.2634   6.8300   2.8353   8.3108   6.9418
  4.6596   1.9511   6.3037   5.1471   7.8076
  3.8592   7.3977   7.2260   1.6832   5.1993
[torch.FloatTensor of size 8x5]


 0.1044  0.0067 -0.0247 -0.0800 -0.1355 -0.0800 -0.0247  0.0067
-0.1948 -0.0138 -0.1396 -0.0084  0.0774  0.0370  0.1352  0.1332
-0.0153 -0.0668  0.1799  0.0574  0.0394  0.1392  0.0268 -0.1462
 0.0199  0.0676 -0.1475 -0.0332  0.1312  0.0740 -0.1128 -0.1948
-0.0416 -0.0159 -0.0166 -0.0070  0.1471 -0.0070 -0.0166 -0.0159
 0.0199 -0.1948 -0.1128  0.0740  0.1312 -0.0332 -0.1475  0.0676
-0.0153 -0.1462  0.0268  0.1392  0.0394  0.0574  0.1799 -0.0668
-0.1948  0.1332  0.1352  0.0370  0.0774 -0.0084 -0.1396 -0.0138
[torch.FloatTensor of size 8x8]



## Parametrized example

This implements a layer with learnable weights.

It implements the Cross-correlation with a learnable kernel.

In deep learning literature, it's confusingly referred to as Convolution.

The backward computes the gradients wrt the input and gradients wrt the filter.

**Implementation:**

*Please Note that the implementation serves as an illustration, and we did not verify it's correctness*

In [40]:
from scipy.signal import convolve2d, correlate2d
from torch.nn.modules.module import Module

class ScipyConv2dFunction(Function):
    
    def forward(self, input, filter):
        result = correlate2d(input.numpy(), filter.numpy(), mode='valid')
        self.save_for_backward(input, filter)
        return torch.FloatTensor(result)
    
    def backward(self, grad_output):
        input, filter = self.saved_tensors
        grad_input = convolve2d(grad_output.numpy(), filter.t().numpy(), mode='full')
        grad_filter = convolve2d(input.numpy(), grad_output.numpy(), mode='valid')
        return torch.FloatTensor(grad_input), torch.FloatTensor(grad_filter)


class ScipyConv2d(Module):
    
    def __init__(self, kh, kw):
        super(ScipyConv2d, self).__init__(
            filter=torch.randn(kh, kw)
        )
    
    def forward(self, input):
        return ScipyConv2dFunction()(input, self.filter)

**Example usage: **

In [41]:
module = ScipyConv2d(3, 3)
print(list(module.parameters()))
input = Variable(torch.randn(10, 10), requires_grad=True)
output = module(input)
print(output)
output.backward(torch.randn(8, 8))
print(input.grad)

[Variable containing:
-1.0235  0.9875  0.2565
 0.1980 -0.6102  0.1088
-0.2887  0.4421  0.4697
[torch.FloatTensor of size 3x3]
]
Variable containing:
 0.7426 -0.4963  2.1839 -0.0167 -1.6349 -0.7259 -0.2989  0.0568
-0.3100  2.2298 -2.2832  0.5753  4.0489  0.1377  0.1672  0.6429
-1.8680  1.3115  1.8970  0.3323 -4.5448 -0.0464 -2.3960  1.5496
-0.6578  0.6759  0.5512 -0.3498  2.6668  1.3984  1.9388 -1.6464
-0.5867  0.5676  2.8697 -0.5566 -2.8876  1.2372 -1.1336 -0.0219
-2.1587  1.1444 -0.5513 -0.5551  1.8229  0.6331 -0.0577 -1.4510
 2.6664  1.4183  2.1640  0.4424 -0.3112 -2.0792  1.7458 -3.3291
-0.4942 -2.1142 -0.2624  0.8993  1.4487  2.1706 -1.4943  0.8073
[torch.FloatTensor of size 8x8]


 0.2528  0.6793  1.4519  0.8932 -1.6100  0.2802  0.7728 -1.7915  0.6271 -0.4103
 1.1033  0.9326 -0.6076  0.0806  2.0530 -1.5469 -0.4001  2.3436 -1.4082  0.6746
-2.2699  0.4997 -1.0990 -0.9396 -2.2007 -0.3414 -1.1383  1.5647 -0.8794  0.9267
-0.0902 -2.0114  1.1145 -1.1107  0.4190 -0.7028  2.7191 -0.6072  